In [1]:
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="embedding-passage")

DB_URI = os.environ["POSTGRES_URI"]     
# from langchain_postgres import PGEngine, PGVectorStore
# pg_engine = PGEngine.from_connection_string(
#     url=DB_URI
# )

from langchain_postgres import PGVector
from langchain_core.documents import Document
vector_store = PGVector(
    embeddings=embeddings,
    collection_name="SPRI_TAG",
    connection=DB_URI
)

In [2]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            name = path.stem.replace("_output_split_documents", "")
            doc.metadata["title"] = name
           
            if name in ["SPRI_2022", "SPRI_2023", "SPRI_2025"]:
                doc.metadata["category"] = "AI-index"
                if name == "SPRI_2022":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI윤리", "AI인재"]
                elif name == "SPRI_2023":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI규제", "AI인재"]
                elif name == "SPRI_2025":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI스타트업", "정부정책"]
            elif "SPRI_is" in name:
                doc.metadata["category"] = "issue-report"
                if name == "SPRI_is_211":
                    doc.metadata["tag"] = ["AI스타트업", "기술사업화", "기업가정신", "AI인재"]
                elif name == "SPRI_is_212":
                    doc.metadata["tag"] = ["SW안전", "기능안전", "오토파일럿"]
                elif name == "SPRI_is_213":
                    doc.metadata["tag"] = ["미래기술", "산업전망", "AI스타트업", "AI인재"]
            elif "SPRI_c" in name:
                doc.metadata["category"] = "column"
                if name == "SPRI_c_1":
                    doc.metadata["tag"] = ["Govtech", "정부혁신", "디지털정부"]
                elif name == "SPRI_c_2":
                    doc.metadata["tag"] = ["기능안전", "산업전망", "AI인재"]
                elif name == "SPRI_c_3":
                    doc.metadata["tag"] = ["산업전망망", "기업가정신", "AI윤리", "AI인재"]
                
                    
    papers.append(split_documents)


In [59]:
papers[3][0].metadata

{'page': 1,
 'image_id': [4],
 'image_path': ['/images/SPRI_c_1_cropped_figure_4.png'],
 'text_summary': '- 주제: GovTech 확산의 두 가지 엔진은 정당성(Legitimacy)과 역능적 행위자성(Empowered Actorhood)이다. 글쓴이는 유두호로 SPRI의 연구원이다.\n\n- GovTech 정의 및 목적: GovTech는 정부(Government)와 기술(Technology)의 결합으로, 공공 서비스 개선과 정부 운영의 효율성 향상을 넘어서 사회 문제 해결 중심의 공공 혁신을 지향한다. 기술 자체보다 이를 행정에 어떻게 접목하고 민간과 어떻게 협업할 것인가에 방점이 찍혀 있으며, 새로운 형태의 거버넌스로 주목받는다.\n\n- 핵심 차이점: 기존 전자정부의 디지털화에 머물지 않고, 사회 문제 해결을 위한 혁신 주도와 민간 협업을 강조한다.\n\n- 시장 전망: 글로벌 GovTech 시장 규모는 2024년 약 6,155.9억 달러로 추정되며, 2033년에는 약 2조 3,050억 달러에 이를 것으로 전망된다. 2025년부터 2033년까지의 CAGR도 상당한 수준으로 예상된다(본문에 제시되나 구체 수치는 생략 가능).\n\n- 참고: 본문은 GovTech의 정의와 시장 규모 추정을 제시하며, 관련 출처로 여러 연구를 인용하고 있다.',
 'image_summary': '다음과 같이 한국어로 묘사할 수 있습니다.\n\n- 그림은 GovTech의 디지털 공공 서비스 혁신을 상징하는 일러스트로 보인다.\n- 중앙에 큰 스마트폰/패널 모양의 기기가 배치되어 있고, 그 위아래로 세 명의 사람들이 서로 자료를 들고 대화하는 모습이다.\n  - 왼쪽 인물: 파란 재킷을 입은 남성이 노트를 들고 프레젠테이션을 하는 듯한 자세.\n  - 위 칸에 앉아 있는 인물: 노트를 들고 있는 남성의 모습으로, 기기 위에 올라앉아 있다.\n  - 오른쪽 인물: 노란 색 치마와 파란 셔츠를 입은 여

In [60]:
len(papers)

9

In [61]:
for paper in papers:
    vector_store.add_documents(paper)

In [55]:
papers[3][0].metadata

{'page': 1,
 'image_id': [4],
 'image_path': ['/images/SPRI_c_1_cropped_figure_4.png'],
 'text_summary': '- 주제: GovTech 확산의 두 가지 엔진은 정당성(Legitimacy)과 역능적 행위자성(Empowered Actorhood)이다. 글쓴이는 유두호로 SPRI의 연구원이다.\n\n- GovTech 정의 및 목적: GovTech는 정부(Government)와 기술(Technology)의 결합으로, 공공 서비스 개선과 정부 운영의 효율성 향상을 넘어서 사회 문제 해결 중심의 공공 혁신을 지향한다. 기술 자체보다 이를 행정에 어떻게 접목하고 민간과 어떻게 협업할 것인가에 방점이 찍혀 있으며, 새로운 형태의 거버넌스로 주목받는다.\n\n- 핵심 차이점: 기존 전자정부의 디지털화에 머물지 않고, 사회 문제 해결을 위한 혁신 주도와 민간 협업을 강조한다.\n\n- 시장 전망: 글로벌 GovTech 시장 규모는 2024년 약 6,155.9억 달러로 추정되며, 2033년에는 약 2조 3,050억 달러에 이를 것으로 전망된다. 2025년부터 2033년까지의 CAGR도 상당한 수준으로 예상된다(본문에 제시되나 구체 수치는 생략 가능).\n\n- 참고: 본문은 GovTech의 정의와 시장 규모 추정을 제시하며, 관련 출처로 여러 연구를 인용하고 있다.',
 'image_summary': '다음과 같이 한국어로 묘사할 수 있습니다.\n\n- 그림은 GovTech의 디지털 공공 서비스 혁신을 상징하는 일러스트로 보인다.\n- 중앙에 큰 스마트폰/패널 모양의 기기가 배치되어 있고, 그 위아래로 세 명의 사람들이 서로 자료를 들고 대화하는 모습이다.\n  - 왼쪽 인물: 파란 재킷을 입은 남성이 노트를 들고 프레젠테이션을 하는 듯한 자세.\n  - 위 칸에 앉아 있는 인물: 노트를 들고 있는 남성의 모습으로, 기기 위에 올라앉아 있다.\n  - 오른쪽 인물: 노란 색 치마와 파란 셔츠를 입은 여

In [ ]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            name = path.stem.replace("_output_split_documents", "")
            doc.metadata["title"] = name
           
            if name in ["SPRI_2022", "SPRI_2023", "SPRI_2025"]:
                doc.metadata["category"] = "AI-index"
                if name == "SPRI_2022":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI윤리", "AI인재"]
                elif name == "SPRI_2023":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI규제", "AI인재"]
                elif name == "SPRI_2025":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI스타트업", "정부정책"]
            elif "SPRI_is" in name:
                doc.metadata["category"] = "issue-report"
                if name == "SPRI_is_211":
                    doc.metadata["tag"] = ["AI스타트업", "기술사업화", "기업가정신", "AI인재"]
                elif name == "SPRI_is_212":
                    doc.metadata["tag"] = ["SW안전", "기능안전", "오토파일럿"]
                elif name == "SPRI_is_213":
                    doc.metadata["tag"] = ["미래기술", "산업전망", "AI스타트업", "AI인재"]
            elif "SPRI_c" in name:
                doc.metadata["category"] = "column"
                if name == "SPRI_c_1":
                    doc.metadata["tag"] = ["Govtech", "정부혁신", "디지털정부"]
                elif name == "SPRI_c_2":
                    doc.metadata["tag"] = ["기능안전", "산업전망", "AI인재"]
                elif name == "SPRI_c_3":
                    doc.metadata["tag"] = ["산업전망", "기업가정신", "AI윤리", "AI인재"]
                
                    
    papers.append(split_documents)

In [60]:
(1,2) in [1,2,3]

False

In [27]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10, "filter": 
{"$and": [
            {"tag": {"$eq": ["AI인재"]}},
            {"tag": {"$ne": ["AI트렌드"]}}
        ]}
})

In [28]:
retriever.invoke("AI 인덱스")

[Document(id='250c30ca-09f7-4d71-b8f3-cf9322068842', metadata={'id': 84, 'tag': ['AI인덱스', 'AI트렌드', 'AI윤리', 'AI인재'], 'page': 42, 'title': 'SPRI_2022', 'category': 'AI-index', 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': []}, page_content='SPRi 이슈리포트 IS-139\n\n\n  \nAI Index 2022의 주요 내용 및 시사점\n\n참고문헌\n====\n\n국외문헌\n====\n\n  \n\n§ Daniel Zhang, Nestor Maslej, Erik Brynjolfsson, John Etchemendy, Terah  \nLyons, James Manyika, Helen Ngo, Juan Carlos Niebles, Michael Sellitto, Ellie  \nSakhaee, Yoav Shoham, Jack Clark, and Raymond Perrault, “The AI Index  \n2022 Annual Report,” AI Index Steering Committee, Stanford Institute for  \nHuman-Centered AI, Stanford University, March 2022\n\n37'),
 Document(id='b80d26ee-b3be-40ed-b722-8d526128c926', metadata={'id': 9, 'tag': ['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재'], 'page': 7, 'title': 'SPRI_2023', 'category': 'AI-index', 'image_id': [41], 'image_path': ['/images/SPRI_2023_cropped_table_41.png'], 'text_summary': '다음은 제공된 텍스트의 요

In [49]:
from langchain_community.retrievers import BM25Retriever

In [9]:
docs = []
for paper in papers:
    docs.extend(paper)

In [10]:
len(docs)

247

In [76]:
from sqlalchemy import create_engine, text

engine = create_engine(DB_URI)
def sql_query_for_BM25() -> list[tuple[str, dict]]:
    with engine.connect() as conn:
        # Custom SQL query
        query = conn.execute(text(f"""
            SELECT 
                e.document,
                e.cmetadata            
            FROM langchain_pg_embedding e
            JOIN langchain_pg_collection c ON e.collection_id = c.uuid
            WHERE c.name = :collection_name
                AND NOT (e.cmetadata->'tag') ?| array[:tag1, :tag2]
                AND e.cmetadata->'tag' ? :tag3
        """), {
            "collection_name": "SPRI_TAG",
            "tag1": "AI트렌드",
            "tag2": "AI인덱스",
            "tag3": "AI인재"
            
        })
        result = query.fetchall()
        arr = []
        for r in result:
            arr.append(Document(page_content=r[0], metadata=r[1]))
        return arr

result_bm25 = sql_query_for_BM25()
bm25_retriever = BM25Retriever.from_documents(result_bm25)
bm25_retriever.k = 10

In [78]:
result_bm25 = bm25_retriever.invoke("AI 기술")

In [77]:
def sql_query_for_PG(query: str) -> list[tuple[str, dict]]:
    query_embedding = embeddings.embed_query(query)
    with engine.connect() as conn:
        # Custom SQL query
        query = conn.execute(text(f"""
            SELECT 
                e.document,
                e.cmetadata,
                e.embedding <=> CAST(:embedding AS vector) as distance            
            FROM langchain_pg_embedding e
            JOIN langchain_pg_collection c ON e.collection_id = c.uuid
            WHERE c.name = :collection_name
                AND NOT (e.cmetadata->'tag') ?| array[:tag1, :tag2]
                AND e.cmetadata->'tag' ? :tag3
            ORDER BY distance
            LIMIT :limit
        """), {
            "embedding": query_embedding,
            "collection_name": "SPRI_TAG",
            "tag1": "AI트렌드",
            "tag2": "AI인덱스",
            "tag3": "AI인재",
            "limit": 10
        })
        result = query.fetchall()
        arr = []
        for r in result:
            arr.append(Document(page_content=r[0], metadata=r[1]))
        return arr

result_pg = sql_query_for_PG("AI 기술")

In [79]:
result_pg

[Document(metadata={'id': 5, 'tag': ['AI스타트업', '기술사업화', '기업가정신', 'AI인재'], 'page': 5, 'title': 'SPRI_is_211', 'category': 'issue-report', 'image_id': [], 'image_path': [], 'text_summary': '다음은 주어진 텍스트의 핵심 내용을 한국어로 요약한 내용입니다.\n\n- AI는 일반목적기술로서 산업‧사회에 큰 변화를 가져올 수 있지만, 가치를 실현하려면 상용화가 필수이며, 성공적 AI 스타트업은 고급 AI 인재의 창업 역량에서 크게 좌우된다.\n- 한국의 AI 인재 정책은 주로 연구자 양성에 집중해 왔으며, 이를 넘어 ‘기업가’로서의 경로를 핵심 축으로 포함시켜야 한다는 제안이 제시된다.\n- 해외(EU, 미국, 중국)와 국내 사례를 분석한 결과, 고급 인재가 기업가 경로를 선택하도록 돕는 정책이 공통 목표이며, 스타트업 인턴십, 대회, 해외 연수, 산학협력 프로젝트, 신기업 지원 등 실무적 경험을 통해 창업 역량을 키우는 정책이 강조된다.\n- 일부 사례는 대학원 교육 과정에 기업가정신(entrepreneurship) 교육을 직접 통합하고, 창업 역량을 주요 성과지표로 삼는 교육 정책을 도입한다.\n- 결론적으로, 창업 지원만으로는 충분치 않으며, 경력 경로를 다각화하고 임무지향적 생태계를 조성하는 방향의 정책이 필요하다. 나아가 ‘기업가형 AI 국가(realizing an entrepreneurial AI state)’를 실현하기 위한 국가 비전과 종합 전략이 제시되어야 한다.', 'image_summary': []}, page_content="SPRi 이슈리포트 IS-211\n\n\n  \n’기업가형 AI 인재‘ 육성의 필요성: AI 강국 도약을 위한 한국형 전략의 출발점\n\nExecutive Summary\n=================\n\nArtificial Intelligence (AI), as a gener

In [80]:
result_bm25

[Document(metadata={'id': 49, 'tag': ['미래기술', '산업전망', 'AI스타트업', 'AI인재'], 'page': 27, 'title': 'SPRI_is_213', 'category': 'issue-report', 'image_id': [280], 'image_path': ['/images/SPRI_is_213_cropped_figure_280.png'], 'text_summary': '요약\n\n- 성장성: 분산 검증·집단 합의·확장 가능한 감시·검증 체계의 채택이 증가하고 규제·감사 요구와 맞물려 확산되며, 엔터프라이즈 레드팀·감사·리스크 관리 워크플로에 결합해 성장 중이다.\n- 불확실성: 분산 구조에서의 책임주체 정의, 합의 메커니즘의 조작·공격 내성, 감사 데이터 접근권 등 법·제도적 불확실성이 크고, 모델/데이터 거버넌스 변화 속도가 빨라 표준이 제정되더라도 지속적인 추적과 업데이트가 필요하다.\n- 영향력: AI 거버넌스 핵심 인프라로서 규제·감사 체계 자동화의 핵심 요소가 되며, 대형 언어모델 안전성 및 AI 위험관리 표준화의 기반이 되는 신뢰성 확보 기술로 자리매김한다.\n- 잠재시장: 글로벌 AI 거버넌스 시장의 핵심 세그먼트로 성장하며, 2030년 약 150~200억 USD 규모로 예상되고 AI 감사·리스크 관리·규제 대응 시장의 30~40%를 차지할 것으로 보인다.\n- 기술체계: 분산 검증자 네트워크, 합의/투표·집계 시스템, 오딧·사고보고 파이프라인, 해석가능성/거버넌스 API, 정책 준수 레이어로 구성된다.\n- 부문 시사: 레드팀·AI 감사 서비스 및 책임형 AI 솔루션의 시장 부상으로 B2B 시장의 확장성이 큼.\n- 그림/계보 맥락: 분산 AI 얼라인먼트의 흐름은 다중 에이전트 시스템, AI-인간 협력, 적응형 LLM 훈련 기술 등 다양한 흐름을 포괄하며, 향후 AI 안전규제·국제 표준화 논의의 중심이 될 가능성이 높다.', 'image_summary': '다음은 이미지의 내용 요약입니다.\n\n-